In [1]:
import warnings
warnings.filterwarnings('ignore')

from astropy.table import Table
import numpy as np
import matplotlib.pyplot as plt
import astropy.constants as ac
import astropy.units as au
from glob import glob
from spectral_cube import SpectralCube

from astropy.io import fits
import matplotlib as mpl
import pyregion
import aplpy
import math
import os
import pickle
from tqdm.auto import tqdm 
import reproject
import seaborn as sns
import gc

from astropy.wcs import WCS
from astropy.coordinates import SkyCoord
from astropy.nddata import Cutout2D

In [2]:
"""Easy pickel"""

def save_pickle(a, filename): 
    with open(filename, 'wb') as handle:
        pickle.dump(a, handle, protocol=pickle.HIGHEST_PROTOCOL)
    print('[INFO] Saved to %s' %filename)
    return()

def load_pickle(filename):
    with open('%s'%filename, 'rb') as handle:
        b = pickle.load(handle)
    print('[INFO] Load %s' %filename)
    return(b)

In [3]:
"""Cut out test region - using spectral cube because thats easy"""

def get_cropmap(hdu_, region):
    
    hdu_tmp = hdu_.copy() #Creating dummy file
    hdu_tmp.data = [hdu_tmp.data, hdu_tmp.data]

    hdu_tmp.header['CDELT3'] = 1
    hdu_tmp.header['CRPIX3'] = 1
    hdu_tmp.header['CRVAL3'] = 1
    hdu_tmp.header['CUNIT3'] = 'km/s'
    hdu_tmp.header['CTYPE3'] = 'VELO'

    cube = SpectralCube.read(hdu_tmp)
    shape_old = cube.hdu.shape
    minax = np.nanmin(shape_old[1:])
    
    cube = cube.subcube_from_ds9region(region)
    cube = cube[:,0:minax,0:minax]
    
    hdu_tmp = cube.hdu 
    hdu_tmp.data = hdu_tmp.data[0]
    del hdu_tmp.header['*3']
    hdu_tmp.header['WCSAXES'] = 2

    hdu_= hdu_tmp
    
    return(hdu_)

In [4]:
# Replace with your file dirs here... 
# 
dir_hst = '../../../data/hst_cropped/'
hst01_file = '%s/ngc628_uvis_f275w_exp_drc_sci.fits' %dir_hst
hst02_file = '%s/ngc628_uvis_f336w_exp_drc_sci.fits' %dir_hst
hst03_file = '%s/ngc628_acs_f435w_exp_drc_sci.fits' %dir_hst
hst04_file = '%s/ngc628_uvis_f555w_exp_drc_sci.fits' %dir_hst
hst05_file = '%s/ngc628_acs_f814w_exp_drc_sci.fits' %dir_hst
hst06_file = '../../../data/hst_contsub/555w814w/ngc628_halpha.fits'

# 
musha_file = '../../../data/muse/maps/HA6562_FLUX.fits'
almaco_file = '../../../data/alma/ngc0628_12m+7m+tp_co21_broad_tpeak.fits'
almaco1_file = '../../../data/alma/ngc0628_12m+7m+tp_co21_broad_mom0.fits'
vlahi_file = '../../../data/vla/NGC_628_NA_MOM0_THINGS.fits'

# 
dir_hst = '../../../data/jwst_updated_v0p4p0/'
jwst01_file = '%s/ngc0628_nircam_lv3_f200w_i2d_align.fits' %dir_hst
jwst02_file = '%s/ngc0628_nircam_lv3_f300m_i2d_align.fits' %dir_hst
jwst03_file = '%s/ngc0628_nircam_lv3_f335m_i2d_align.fits' %dir_hst
jwst04_file = '%s/ngc0628_nircam_lv3_f360m_i2d_align.fits' %dir_hst

dir_hst = '../../../data/jwst_updated_v0p5_miri/'
jwst05_file = '%s/ngc0628_miri_f770w_anchored.fits' %dir_hst
jwst06_file = '%s/ngc0628_miri_f1000w_anchored.fits' %dir_hst
jwst07_file = '%s/ngc0628_miri_f1130w_anchored.fits' %dir_hst
jwst08_file = '%s/ngc0628_miri_f2100w_anchored.fits' %dir_hst


dir_hst = '../../../data/jwst_feast/'
jwst09_file = '%s/jw01783-o004_t008_nircam_clear-f115w_i2d_J2000.fits' %dir_hst
jwst10_file = '%s/jw01783-o004_t008_nircam_clear-f150w_i2d_J2000.fits' %dir_hst
jwst11_file = '%s/jw01783-o004_t008_nircam_clear-f187n_i2d_J2000.fits' %dir_hst
jwst12_file = '%s/jw01783-o004_t008_nircam_clear-f187n_ONLY_J2000.fits' %dir_hst
jwst13_file = '%s/jw01783-o004_t008_nircam_clear-f277w_i2d_J2000.fits' %dir_hst
jwst14_file = '%s/jw01783-o004_t008_nircam_clear-f444w_i2d_J2000.fits' %dir_hst
jwst15_file = '%s/jw01783-o004_t008_nircam_f405_ONLY_J2000.fits' %dir_hst
jwst16_file = '%s/jw01783-o004_t008_nircam_f405n-f444w_i2d_J2000.fits' %dir_hst

asto_file = '../../../data/astrosat/NGC0628_FUV_ave.fits'
spit_file = '../../../data/spitzer/NGC_0628_IRAC_I4_bms2014.fits'

# Loading catalouges
muscat_file = '../../../data/muse/catalouge/NGC0628_nebulae_mask_V2.fits' #Muse Neb catalouge

# 
dir_asscat = '../../../data/cluster_catalogs/ngc628-vselect/'
asscat1_file = '%s/ws16pc/PHANGS_IR3_hst_wfc3_ngc628_v1p3_multi_assoc-vselect-ws16pc-idmask.fits' %dir_asscat #HST association catalogue
asscat2_file = '%s/ws32pc/PHANGS_IR3_hst_wfc3_ngc628_v1p3_multi_assoc-vselect-ws32pc-idmask.fits' %dir_asscat #HST association catalogue
asscat3_file = '%s/ws64pc/PHANGS_IR3_hst_wfc3_ngc628_v1p3_multi_assoc-vselect-ws64pc-idmask.fits' %dir_asscat #HST association catalogue

In [5]:
# # loading files
# # changing everything into log scale
hst01_hdu = fits.open(hst01_file)[0]
hst02_hdu = fits.open(hst02_file)[0]
hst03_hdu = fits.open(hst03_file)[0]
hst04_hdu = fits.open(hst04_file)[0]
hst05_hdu = fits.open(hst05_file)[0]
hst06_hdu = fits.open(hst06_file)[0]

# 
musha_hdu = fits.open(musha_file)[1]
musha_hdu.data[np.isnan(musha_hdu.data)] = -100

almaco_hdu = fits.open(almaco_file)[0]
almaco1_hdu = fits.open(almaco1_file)[0]

vlahi_hdu = fits.open(vlahi_file)[0]
vlahi_hdu.data = np.squeeze(vlahi_hdu.data)
del vlahi_hdu.header['*3*']
del vlahi_hdu.header['*4*']

#
jwst01_hdu = fits.open(jwst01_file)[1]
jwst02_hdu = fits.open(jwst02_file)[1]
jwst03_hdu = fits.open(jwst03_file)[1]
jwst04_hdu = fits.open(jwst04_file)[1]
jwst05_hdu = fits.open(jwst05_file)[0]
jwst06_hdu = fits.open(jwst06_file)[0]
jwst07_hdu = fits.open(jwst07_file)[0]
jwst08_hdu = fits.open(jwst08_file)[0]

jwst01_hdu.data[jwst01_hdu.data == 0] = np.nan
jwst02_hdu.data[jwst02_hdu.data == 0] = np.nan
jwst03_hdu.data[jwst03_hdu.data == 0] = np.nan
jwst04_hdu.data[jwst04_hdu.data == 0] = np.nan
jwst05_hdu.data[jwst05_hdu.data == 0] = np.nan
jwst06_hdu.data[jwst06_hdu.data == 0] = np.nan
jwst07_hdu.data[jwst07_hdu.data == 0] = np.nan
jwst08_hdu.data[jwst08_hdu.data == 0] = np.nan

jwst09_hdu = fits.open(jwst09_file)[0]
jwst10_hdu = fits.open(jwst10_file)[0]
jwst11_hdu = fits.open(jwst11_file)[0]
jwst12_hdu = fits.open(jwst12_file)[0]
jwst13_hdu = fits.open(jwst13_file)[0]
jwst14_hdu = fits.open(jwst14_file)[0]
jwst15_hdu = fits.open(jwst15_file)[0]
jwst16_hdu = fits.open(jwst16_file)[0]

jwst09_hdu.data[jwst09_hdu.data == 0] = np.nan
jwst10_hdu.data[jwst10_hdu.data == 0] = np.nan
jwst11_hdu.data[jwst11_hdu.data == 0] = np.nan
jwst12_hdu.data[jwst12_hdu.data == 0] = np.nan
jwst13_hdu.data[jwst13_hdu.data == 0] = np.nan
jwst14_hdu.data[jwst14_hdu.data == 0] = np.nan
jwst15_hdu.data[jwst15_hdu.data == 0] = np.nan
jwst16_hdu.data[jwst16_hdu.data == 0] = np.nan

#
asto_hdu = fits.open(asto_file)[0]
spit_hdu = fits.open(spit_file)[0]

data, _ = reproject.reproject_interp(spit_hdu, musha_hdu.header)
spit_hdu = fits.PrimaryHDU(data, musha_hdu.header)

# loading catalouges
muscat_hdu = fits.open(muscat_file)[0]
asscat1_hdu = fits.open(asscat1_file)[0]
asscat2_hdu = fits.open(asscat2_file)[0]
asscat3_hdu = fits.open(asscat3_file)[0]

muscat_hdu.data = np.array(muscat_hdu.data, dtype=float)
muscat_hdu.data[muscat_hdu.data==-1] = np.nan

asscat1_hdu = fits.PrimaryHDU(reproject.reproject_interp(asscat1_hdu, hst01_hdu.header, return_footprint=False, order='nearest-neighbor'), hst01_hdu.header)
asscat2_hdu = fits.PrimaryHDU(reproject.reproject_interp(asscat2_hdu, hst01_hdu.header, return_footprint=False, order='nearest-neighbor'), hst01_hdu.header)
asscat3_hdu = fits.PrimaryHDU(reproject.reproject_interp(asscat3_hdu, hst01_hdu.header, return_footprint=False, order='nearest-neighbor'), hst01_hdu.header)

asscat1_hdu.data[asscat1_hdu.data==0] = np.nan
asscat2_hdu.data[asscat2_hdu.data==0] = np.nan
asscat3_hdu.data[asscat3_hdu.data==0] = np.nan

INFO: 
                Inconsistent SIP distortion information is present in the FITS header and the WCS object:
                SIP coefficients were detected, but CTYPE is missing a "-SIP" suffix.
                astropy.wcs is using the SIP distortion coefficients,
                therefore the coordinates calculated here might be incorrect.

                If you do not want to apply the SIP distortion coefficients,
                please remove the SIP coefficients from the FITS header or the
                WCS object.  As an example, if the image is already distortion-corrected
                (e.g., drizzled) then distortion components should not apply and the SIP
                coefficients should be removed.

                While the SIP distortion coefficients are being applied here, if that was indeed the intent,
                for consistency please append "-SIP" to the CTYPE in the FITS header or the WCS object.

                 [astropy.wcs.wcs]


In [6]:
hst01_hdu.data = np.array(hst01_hdu.data, dtype='float32')
hst02_hdu.data = np.array(hst02_hdu.data, dtype='float32')
hst03_hdu.data = np.array(hst03_hdu.data, dtype='float32')
hst04_hdu.data = np.array(hst04_hdu.data, dtype='float32')
hst05_hdu.data = np.array(hst05_hdu.data, dtype='float32')
hst06_hdu.data = np.array(hst06_hdu.data, dtype='float32')
musha_hdu.data = np.array(musha_hdu.data, dtype='float32')
almaco_hdu.data = np.array(almaco_hdu.data, dtype='float32')
almaco1_hdu.data = np.array(almaco1_hdu.data, dtype='float32')
vlahi_hdu.data = np.array(vlahi_hdu.data, dtype='float32')
jwst01_hdu.data = np.array(jwst01_hdu.data, dtype='float32')
jwst02_hdu.data = np.array(jwst02_hdu.data, dtype='float32')
jwst03_hdu.data = np.array(jwst03_hdu.data, dtype='float32')
jwst04_hdu.data = np.array(jwst04_hdu.data, dtype='float32')
jwst05_hdu.data = np.array(jwst05_hdu.data, dtype='float32')
jwst06_hdu.data = np.array(jwst06_hdu.data, dtype='float32')
jwst07_hdu.data = np.array(jwst07_hdu.data, dtype='float32')
jwst08_hdu.data = np.array(jwst08_hdu.data, dtype='float32')
jwst09_hdu.data = np.array(jwst09_hdu.data, dtype='float32')
jwst10_hdu.data = np.array(jwst10_hdu.data, dtype='float32')
jwst11_hdu.data = np.array(jwst11_hdu.data, dtype='float32')
jwst12_hdu.data = np.array(jwst12_hdu.data, dtype='float32')
jwst13_hdu.data = np.array(jwst13_hdu.data, dtype='float32')
jwst14_hdu.data = np.array(jwst14_hdu.data, dtype='float32')
jwst15_hdu.data = np.array(jwst15_hdu.data, dtype='float32')
jwst16_hdu.data = np.array(jwst16_hdu.data, dtype='float32')
asto_hdu.data = np.array(asto_hdu.data, dtype='float32')
spit_hdu.data = np.array(spit_hdu.data, dtype='float32')
muscat_hdu.data = np.array(muscat_hdu.data, dtype='float32')
asscat1_hdu.data = np.array(asscat1_hdu.data, dtype='float32')
asscat2_hdu.data = np.array(asscat2_hdu.data, dtype='float32')
asscat3_hdu.data = np.array(asscat3_hdu.data, dtype='float32')

In [7]:
def get_regions(regions_file, hdu):
    
    regions = pyregion.open(regions_file).as_imagecoord(hdu.header)
    n = len(regions)
    
    ra = np.empty(n)*np.nan*au.deg
    dec = np.empty(n)*np.nan*au.deg
    width = np.empty(n)*np.nan*au.deg
    height = np.empty(n)*np.nan*au.deg
    position = ['']*n
    
    for i, region in enumerate(regions):

        ra[i] = float(region.params[0].text)*au.deg
        dec[i] = float(region.params[1].text)*au.deg
        width[i] = region.params[2].degree*au.deg
        height[i] = region.params[3].degree*au.deg
        
    position = SkyCoord(ra=ra, dec=dec, frame='icrs')

    return({'ra':ra, 'dec':dec, 'width':width, 'height':height, 'position': position})

def get_croppeddata(hdu, i, regions, square=True):

    hdu_crop = hdu.copy()
    del hdu
    _ = gc.collect()
    
    wcs = WCS(hdu_crop)
    
    if square: 
        radius = max([regions['width'][i], regions['height'][i]]) *1.3 ##Give some buffer for plotting
        cutout = Cutout2D(hdu_crop.data, 
                          regions['position'][i], 
                          radius, 
                          wcs=wcs)
    else: 
        cutout = Cutout2D(hdu_crop.data, 
                      regions['position'][i], 
                      [regions['width'][i], regions['height'][i]], 
                      wcs=wcs)
    
    hdu_crop.data = cutout.data
    hdu_crop.header.update(cutout.wcs.to_header())
    
    del cutout
    _ = gc.collect()
    
    return(hdu_crop)

def get_croppeddata_parallel(hdu, regions, n_jobs=-1):
    
    from joblib import Parallel, delayed
    
    n = len(regions['ra'])

    hdus = Parallel(n_jobs=n_jobs, backend='threading')(delayed(get_croppeddata)(i,j,k) for i,j,k in zip([hdu]*n, range(n), [regions]*n))
    
    return(hdus)

In [8]:
regions_file = './../../analysis/catalogue/sample_v2.reg'
regions = get_regions(regions_file, hst01_hdu)

n = len(regions['ra'])
print('[INFO] Number of HII regions in sample --> %i' %n)

[INFO] Number of HII regions in sample --> 215


In [9]:
# hdus = get_croppeddata_parallel(hst01_hdu, regions)
# save_pickle(hdus, '../../analysis/catalogue/hst01_hdus.pickel')
# del hdus
# _ = gc.collect()

# hdus = get_croppeddata_parallel(hst02_hdu, regions)
# save_pickle(hdus, '../../analysis/catalogue/hst02_hdus.pickel') 
# del hdus
# _ = gc.collect()

# hdus = get_croppeddata_parallel(hst03_hdu, regions)
# save_pickle(hdus, '../../analysis/catalogue/hst03_hdus.pickel') 
# del hdus
# _ = gc.collect()

# hdus = get_croppeddata_parallel(hst04_hdu, regions)
# save_pickle(hdus, '../../analysis/catalogue/hst04_hdus.pickel') 
# del hdus
# _ = gc.collect()

# hdus = get_croppeddata_parallel(hst05_hdu, regions)
# save_pickle(hdus, '../../analysis/catalogue/hst05_hdus.pickel') 
# del hdus
# _ = gc.collect()

# hdus = get_croppeddata_parallel(hst06_hdu, regions)
# save_pickle(hdus, '../../analysis/catalogue/hst06_hdus.pickel') 
# del hdus
# _ = gc.collect()

# hdus = get_croppeddata_parallel(musha_hdu, regions)
# save_pickle(hdus, '../../analysis/catalogue/musha_hdus.pickel')
# del hdus
# _ = gc.collect()

# hdus = get_croppeddata_parallel(almaco_hdu, regions)
# save_pickle(hdus, '../../analysis/catalogue/almaco_hdus.pickel')
# del hdus
# _ = gc.collect()

# hdus = get_croppeddata_parallel(almaco1_hdu, regions)
# save_pickle(hdus, '../../analysis/catalogue/almaco1_hdus.pickel')
# del hdus
# _ = gc.collect()

# hdus = get_croppeddata_parallel(vlahi_hdu, regions)
# save_pickle(hdus, '../../analysis/catalogue/vlahi_hdus.pickel')
# del hdus
# _ = gc.collect()

# hdus = get_croppeddata_parallel(jwst01_hdu, regions)
# save_pickle(hdus, '../../analysis/catalogue/jwst01_hdus.pickel')
# del hdus
# _ = gc.collect()

# hdus = get_croppeddata_parallel(jwst02_hdu, regions)
# save_pickle(hdus, '../../analysis/catalogue/jwst02_hdus.pickel') 
# del hdus
# _ = gc.collect()

# hdus = get_croppeddata_parallel(jwst03_hdu, regions)
# save_pickle(hdus, '../../analysis/catalogue/jwst03_hdus.pickel') 
# del hdus
# _ = gc.collect()

# hdus = get_croppeddata_parallel(jwst04_hdu, regions)
# save_pickle(hdus, '../../analysis/catalogue/jwst04_hdus.pickel') 
# del hdus
# _ = gc.collect()

# hdus = get_croppeddata_parallel(jwst05_hdu, regions)
# save_pickle(hdus, '../../analysis/catalogue/jwst05_hdus.pickel') 
# del hdus
# _ = gc.collect()

# hdus = get_croppeddata_parallel(jwst06_hdu, regions)
# save_pickle(hdus, '../../analysis/catalogue/jwst06_hdus.pickel') 
# del hdus
# _ = gc.collect()

# hdus = get_croppeddata_parallel(jwst07_hdu, regions)
# save_pickle(hdus, '../../analysis/catalogue/jwst07_hdus.pickel') 
# del hdus
# _ = gc.collect()

# hdus = get_croppeddata_parallel(jwst08_hdu, regions)
# save_pickle(hdus, '../../analysis/catalogue/jwst08_hdus.pickel') 
# del hdus
# _ = gc.collect()

# hdus = get_croppeddata_parallel(jwst09_hdu, regions)
# save_pickle(hdus, '../../analysis/catalogue/jwst09_hdus.pickel') 
# del hdus
# _ = gc.collect()

# hdus = get_croppeddata_parallel(jwst10_hdu, regions)
# save_pickle(hdus, '../../analysis/catalogue/jwst10_hdus.pickel') 
# del hdus
# _ = gc.collect()

# hdus = get_croppeddata_parallel(jwst11_hdu, regions)
# save_pickle(hdus, '../../analysis/catalogue/jwst11_hdus.pickel') 
# del hdus
# _ = gc.collect()

# hdus = get_croppeddata_parallel(jwst12_hdu, regions)
# save_pickle(hdus, '../../analysis/catalogue/jwst12_hdus.pickel') 
# del hdus
# _ = gc.collect()

# hdus = get_croppeddata_parallel(jwst13_hdu, regions)
# save_pickle(hdus, '../../analysis/catalogue/jwst13_hdus.pickel') 
# del hdus
# _ = gc.collect()

# hdus = get_croppeddata_parallel(jwst14_hdu, regions)
# save_pickle(hdus, '../../analysis/catalogue/jwst14_hdus.pickel') 
# del hdus
# _ = gc.collect()

# hdus = get_croppeddata_parallel(jwst15_hdu, regions)
# save_pickle(hdus, '../../analysis/catalogue/jwst15_hdus.pickel') 
# del hdus
# _ = gc.collect()

# hdus = get_croppeddata_parallel(jwst16_hdu, regions)
# save_pickle(hdus, '../../analysis/catalogue/jwst16_hdus.pickel') 
# del hdus
# _ = gc.collect()

# hdus = get_croppeddata_parallel(muscat_hdu, regions)
# save_pickle(hdus, '../../analysis/catalogue/muscat_hdus.pickel')
# del hdus

# hdus = get_croppeddata_parallel(asto_hdu, regions)
# save_pickle(hdus, '../../analysis/catalogue/asto_hdus.pickel') 
# del hdus
# _ = gc.collect()

# hdus = get_croppeddata_parallel(spit_hdu, regions)
# save_pickle(hdus, '../../analysis/catalogue/spit_hdus.pickel') 
# del hdus
# _ = gc.collect()

hdus = get_croppeddata_parallel(asscat1_hdu, regions)
save_pickle(hdus, '../../analysis/catalogue/asscat1_hdus.pickel') 
del hdus
_ = gc.collect()

hdus = get_croppeddata_parallel(asscat2_hdu, regions)
save_pickle(hdus, '../../analysis/catalogue/asscat2_hdus.pickel') 
del hdus
_ = gc.collect()

hdus = get_croppeddata_parallel(asscat3_hdu, regions)
save_pickle(hdus, '../../analysis/catalogue/asscat3_hdus.pickel') 
del hdus
_ = gc.collect()

[INFO] Saved to ../../analysis/catalogue/asscat1_hdus.pickel
[INFO] Saved to ../../analysis/catalogue/asscat2_hdus.pickel
[INFO] Saved to ../../analysis/catalogue/asscat3_hdus.pickel


In [10]:
hst01_hdus = load_pickle('../../analysis/catalogue/hst01_hdus.pickel')
hst02_hdus = load_pickle('../../analysis/catalogue/hst02_hdus.pickel')
hst03_hdus = load_pickle('../../analysis/catalogue/hst03_hdus.pickel')
hst04_hdus = load_pickle('../../analysis/catalogue/hst04_hdus.pickel')
hst05_hdus = load_pickle('../../analysis/catalogue/hst05_hdus.pickel')
hst06_hdus = load_pickle('../../analysis/catalogue/hst06_hdus.pickel')
musha_hdus = load_pickle('../../analysis/catalogue/musha_hdus.pickel')
almaco_hdus = load_pickle('../../analysis/catalogue/almaco_hdus.pickel')
almaco1_hdus = load_pickle('../../analysis/catalogue/almaco1_hdus.pickel')
vlahi_hdus = load_pickle('../../analysis/catalogue/vlahi_hdus.pickel')
jwst01_hdus = load_pickle('../../analysis/catalogue/jwst01_hdus.pickel')
jwst02_hdus = load_pickle('../../analysis/catalogue/jwst02_hdus.pickel')
jwst03_hdus = load_pickle('../../analysis/catalogue/jwst03_hdus.pickel')
jwst04_hdus = load_pickle('../../analysis/catalogue/jwst04_hdus.pickel')
jwst05_hdus = load_pickle('../../analysis/catalogue/jwst05_hdus.pickel')
jwst06_hdus = load_pickle('../../analysis/catalogue/jwst06_hdus.pickel')
jwst07_hdus = load_pickle('../../analysis/catalogue/jwst07_hdus.pickel')
jwst08_hdus = load_pickle('../../analysis/catalogue/jwst08_hdus.pickel')
jwst09_hdus = load_pickle('../../analysis/catalogue/jwst09_hdus.pickel')
jwst10_hdus = load_pickle('../../analysis/catalogue/jwst10_hdus.pickel')
jwst11_hdus = load_pickle('../../analysis/catalogue/jwst11_hdus.pickel')
jwst12_hdus = load_pickle('../../analysis/catalogue/jwst12_hdus.pickel')
jwst13_hdus = load_pickle('../../analysis/catalogue/jwst13_hdus.pickel')
jwst14_hdus = load_pickle('../../analysis/catalogue/jwst14_hdus.pickel')
jwst15_hdus = load_pickle('../../analysis/catalogue/jwst15_hdus.pickel')
jwst16_hdus = load_pickle('../../analysis/catalogue/jwst16_hdus.pickel')
asto_hdus = load_pickle('../../analysis/catalogue/asto_hdus.pickel')
spit_hdus = load_pickle('../../analysis/catalogue/spit_hdus.pickel')
muscat_hdus = load_pickle('../../analysis/catalogue/muscat_hdus.pickel')
asscat1_hdus = load_pickle('../../analysis/catalogue/asscat1_hdus.pickel')
asscat2_hdus = load_pickle('../../analysis/catalogue/asscat2_hdus.pickel')
asscat3_hdus = load_pickle('../../analysis/catalogue/asscat3_hdus.pickel')

hdus = {'hst01_hdus' : hst01_hdus,
        'hst02_hdus' : hst02_hdus,
        'hst03_hdus' : hst03_hdus,
        'hst04_hdus' : hst04_hdus,
        'hst05_hdus' : hst05_hdus, 
        'hst06_hdus' : hst06_hdus,
        'musha_hdus' : musha_hdus,
        'almaco_hdus' : almaco_hdus,
        'almaco1_hdus' : almaco1_hdus,
        'vlahi_hdus' : vlahi_hdus ,
        'jwst01_hdus' : jwst01_hdus, 
        'jwst02_hdus' : jwst02_hdus,
        'jwst03_hdus' : jwst03_hdus,
        'jwst04_hdus' : jwst04_hdus,
        'jwst05_hdus' : jwst05_hdus,
        'jwst06_hdus' : jwst06_hdus,
        'jwst07_hdus' : jwst07_hdus,
        'jwst08_hdus' : jwst08_hdus,
        'jwst09_hdus' : jwst09_hdus,
        'jwst10_hdus' : jwst10_hdus,
        'jwst11_hdus' : jwst11_hdus,
        'jwst12_hdus' : jwst12_hdus,
        'jwst13_hdus' : jwst13_hdus,
        'jwst14_hdus' : jwst14_hdus,
        'jwst15_hdus' : jwst15_hdus,
        'jwst16_hdus' : jwst16_hdus,
        'asto_hdus' : asto_hdus,
        'spit_hdus' : spit_hdus,
        'muscat_hdus' : muscat_hdus,
        'asscat1_hdus' : asscat1_hdus,
        'asscat2_hdus' : asscat2_hdus,
        'asscat3_hdus' : asscat3_hdus}

save_pickle(hdus, '../../analysis/catalogue/hdus_sample_v2.pickel') 

[INFO] Load ../../analysis/catalogue/hst01_hdus.pickel
[INFO] Load ../../analysis/catalogue/hst02_hdus.pickel
[INFO] Load ../../analysis/catalogue/hst03_hdus.pickel
[INFO] Load ../../analysis/catalogue/hst04_hdus.pickel
[INFO] Load ../../analysis/catalogue/hst05_hdus.pickel
[INFO] Load ../../analysis/catalogue/hst06_hdus.pickel
[INFO] Load ../../analysis/catalogue/musha_hdus.pickel
[INFO] Load ../../analysis/catalogue/almaco_hdus.pickel
[INFO] Load ../../analysis/catalogue/almaco1_hdus.pickel
[INFO] Load ../../analysis/catalogue/vlahi_hdus.pickel
[INFO] Load ../../analysis/catalogue/jwst01_hdus.pickel
[INFO] Load ../../analysis/catalogue/jwst02_hdus.pickel
[INFO] Load ../../analysis/catalogue/jwst03_hdus.pickel
[INFO] Load ../../analysis/catalogue/jwst04_hdus.pickel
[INFO] Load ../../analysis/catalogue/jwst05_hdus.pickel
[INFO] Load ../../analysis/catalogue/jwst06_hdus.pickel
[INFO] Load ../../analysis/catalogue/jwst07_hdus.pickel
[INFO] Load ../../analysis/catalogue/jwst08_hdus.pickel

()